# Imports and Set-up

In [ ]:
# Standard Imports
import sys
import pandas as pd
import geopandas as gpd
from shapely.wkt import loads
from tqdm import tqdm

In [ ]:
# Util imports
sys.path.append("../../")  # include parent directory
from src.settings import GPKG_DATA_DIR, TMP_OUT_DIR
from src.duckdb_utils import create_default_connection

## Filepaths

In [ ]:
slope_fpath = GPKG_DATA_DIR / "reclass_slope_linework.gpkg"
elev_fpath = GPKG_DATA_DIR / "caraga-davao-reclassified-elevation.gpkg"
strata_fpath = GPKG_DATA_DIR / "BL-PL" / "caraga-davao-strata-fixed.gpkg"
geotype_fpath = GPKG_DATA_DIR / "BL-PL" / "caraga-davao-ultramafic-rocks.gpkg"
proxy_fpath = GPKG_DATA_DIR / "BL-PL" / "davao_CADT_mining.gpkg"
hrp_fpath = (
    GPKG_DATA_DIR / "BL-PL" / "caraga-davao-gee_deforestation_activity_data.gpkg"
)
hrp2_fpath = (
    GPKG_DATA_DIR
    / "BL-PL"
    / "caraga-davao-gee_deforestation_activity_data_2nd_half_hrp.gpkg"
)
output_slope_fpath = GPKG_DATA_DIR / "davao_proxy_slope.gpkg"
output_elevation_fpath = GPKG_DATA_DIR / "davao-proxy-elevation.gpkg"

## Create/open duckdb instance

In [ ]:
# Connect to the duckdb database
db = create_default_connection(filepath=str(TMP_OUT_DIR / "proxy_area.db"))

In [ ]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
db.execute(query).df()

## Read GPKGs

In [ ]:
slope_reclass = gpd.read_file(slope_fpath)
proxy = gpd.read_file(proxy_fpath)
elev_reclass = gpd.read_file(elev_fpath)
geo = gpd.read_file(geotype_fpath)
strata_gdf = gpd.read_file(strata_fpath)
hrp = gpd.read_file(hrp_fpath)

## Proxy Area

In [ ]:
proxy.head(2)

In [ ]:
proxy.shape

In [ ]:
proxy.CADT_No = proxy.CADT_No.fillna(0)

In [ ]:
proxy = proxy.reset_index()

In [ ]:
proxy["uid"] = (
    proxy["index"].astype(str)
    + "-"
    + proxy["CADT_No"].astype(int).astype(str)
    + "-"
    + proxy["ID_CODE"]
)

In [ ]:
proxy_df = proxy.copy()

In [ ]:
proxy_df["geometry"] = proxy_df["geometry"].to_wkt()

In [ ]:
proxy_df.columns

In [ ]:
query = """
CREATE OR REPLACE TABLE proxy AS 
SELECT 
    uid,
    CADT_No,
    Davao_CADT_No,
    HOLDER,
    TENEMENT_N, 
    DATE_FILED, 
    DATE_APPRO, 
    COMMODITY, 
    REMARKS_2,
    EXPIRYDATE,
    ST_GeomFromText(geometry) as geometry FROM proxy_df
"""

db.execute(query)

In [ ]:
total_area = proxy.to_crs("EPSG:3857").geometry.area.sum() / 10_000

In [ ]:
proxy["area_ha"] = proxy.to_crs("EPSG:3857").geometry.area / 10_000

In [ ]:
tenement_area = proxy[["TENEMENT_N", "area_ha"]].groupby("TENEMENT_N").sum()

In [ ]:
tenement_area

In [ ]:
proxy.Davao_CADT_No.nunique()

## Slope

In [ ]:
slope_reclass.head(2)

In [ ]:
slope_reclass_df = slope_reclass.copy()

In [ ]:
slope_reclass_df["geometry"] = slope_reclass_df["geometry"].to_wkt()

In [ ]:
query = """
CREATE OR REPLACE TABLE slope AS 
SELECT 
    DN,
    ST_GeomFromText(geometry) as geometry FROM slope_reclass_df
"""

db.execute(query)

In [ ]:
query = """ 
CREATE OR REPLACE TABLE proxy_slope AS 
SELECT
    uid,
    DN,
    CADT_No,
    Davao_CADT_No,
    HOLDER,
    TENEMENT_N, 
    DATE_FILED, 
    DATE_APPRO, 
    COMMODITY, 
    REMARKS_2,
    EXPIRYDATE,
    ST_INTERSECTION(proxy.geometry, slope.geometry) as geometry
FROM
    proxy 
JOIN 
    slope
ON 
    ST_INTERSECTS(proxy.geometry, slope.geometry)"""

db.execute(query).df()

In [ ]:
query = """ 
SELECT
   *EXCLUDE(geometry),
   ST_astext(geometry) as geometry
FROM
    proxy_slope 
"""

proxy_slope = db.execute(query).df()

In [ ]:
proxy_slope.head()

In [ ]:
proxy_slope_gdf = proxy_slope.copy()

In [ ]:
proxy_slope_gdf["geometry"] = proxy_slope_gdf["geometry"]

In [ ]:
proxy_slope_gdf["geometry"] = proxy_slope_gdf["geometry"].apply(lambda x: loads(x))
proxy_slope_gdf = gpd.GeoDataFrame(
    proxy_slope_gdf, geometry="geometry", crs="EPSG:4326"
)

In [ ]:
proxy_slope_gdf.to_crs("EPSG:3857", inplace=True)

In [ ]:
proxy_slope_gdf["area_ha"] = proxy_slope_gdf.geometry.area / 10_000

In [ ]:
slope_profile = (
    proxy_slope_gdf[["TENEMENT_N", "DN", "area_ha"]].groupby(["DN", "TENEMENT_N"]).sum()
)

In [ ]:
slope_profile_mng = slope_profile.join(tenement_area, lsuffix="_slope")

In [ ]:
slope_profile_mng["perc"] = (
    slope_profile["area_ha"] / slope_profile_mng["area_ha"]
) * 100

In [ ]:
slope_profile_mng.reset_index(inplace=True)
slope_profile_mng.set_index("TENEMENT_N", inplace=True)

In [ ]:
slope_profile_mng.rename(columns={"DN": "slope_dn"}, inplace=True)

In [ ]:
slope_profile_mng.reset_index(inplace=True)

In [ ]:
slope_profile_mng = (
    slope_profile_mng.pivot(index="TENEMENT_N", columns="slope_dn", values="perc")
    .fillna(0)
    .reset_index()
)

In [ ]:
slope_profile_mng

## Elevation

In [ ]:
elev_reclass_df = elev_reclass.copy()

In [ ]:
elev_reclass_df["geometry"] = elev_reclass_df["geometry"].to_wkt()

In [ ]:
query = """
CREATE OR REPLACE TABLE elev AS 
SELECT 
    DN,
    ST_GeomFromText(geometry) as geometry FROM elev_reclass_df
"""

db.execute(query)

In [ ]:
query = """ 
CREATE OR REPLACE TABLE proxy_elev AS
SELECT
    uid,
    CADT_No,
    Davao_CADT_No,
    HOLDER,
    TENEMENT_N, 
    DATE_FILED, 
    DATE_APPRO, 
    COMMODITY, 
    REMARKS_2,
    EXPIRYDATE,
    elev.DN as elev_dn,
    ST_INTERSECTION(proxy.geometry, elev.geometry) as geometry
FROM
    proxy
JOIN 
    elev
ON 
    ST_INTERSECTS(proxy.geometry, elev.geometry)"""

db.execute(query).df()

In [ ]:
query = """ 
SELECT
   *EXCLUDE(geometry),
   ST_astext(geometry) as geometry
FROM
    proxy_elev 
"""

proxy_elev = db.execute(query).df()

In [ ]:
proxy_elev.head(2)

In [ ]:
proxy_elev_gdf = proxy_elev.copy()

In [ ]:
proxy_elev_gdf["geometry"] = proxy_elev_gdf["geometry"].apply(lambda x: loads(x))
proxy_elev_gdf = gpd.GeoDataFrame(proxy_elev_gdf, geometry="geometry", crs="EPSG:4326")

In [ ]:
proxy_elev_gdf.to_crs("EPSG:3857", inplace=True)

In [ ]:
proxy_elev_gdf["area_ha"] = proxy_elev_gdf.geometry.area / 10_000

In [ ]:
elev_profile = (
    proxy_elev_gdf[["elev_dn", "TENEMENT_N", "area_ha"]]
    .groupby(["elev_dn", "TENEMENT_N"])
    .sum()
)

In [ ]:
elev_profile_mng = elev_profile.join(tenement_area, lsuffix="_elev")

In [ ]:
elev_profile_mng["perc"] = (
    elev_profile_mng["area_ha_elev"] / elev_profile_mng["area_ha"]
) * 100

In [ ]:
elev_profile_mng.reset_index(inplace=True)

In [ ]:
elev_profile_mng = (
    elev_profile_mng.pivot(index="TENEMENT_N", columns="elev_dn", values="perc")
    .fillna(0)
    .reset_index()
)

## Geology

In [ ]:
geo_df = geo.copy()

In [ ]:
geo_df["geometry"] = geo["geometry"].to_wkt()

In [ ]:
geo_df.head()

In [ ]:
query = """
CREATE OR REPLACE TABLE geotype AS 
SELECT 
    *EXCLUDE(geometry),
    ST_GeomFromText(geometry) as geometry FROM geo_df
"""

db.execute(query)

In [ ]:
query = """ 
CREATE OR REPLACE TABLE proxy_geo AS
SELECT
    uid,
    CADT_No,
    Davao_CADT_No,
    HOLDER,
    TENEMENT_N, 
    DATE_FILED, 
    DATE_APPRO, 
    COMMODITY, 
    REMARKS_2,
    EXPIRYDATE,
    Litho,
    remarks,
    ST_INTERSECTION(proxy.geometry, geotype.geometry) as geometry
FROM
    proxy
JOIN 
    geotype
ON 
    ST_INTERSECTS(proxy.geometry, geotype.geometry)"""

db.execute(query).df()

In [ ]:
query = """ 
SELECT
   *EXCLUDE(geometry),
   ST_astext(geometry) as geometry
FROM
    proxy_geo
"""

proxy_geo = db.execute(query).df()

In [ ]:
proxy_geo.head(2)

In [ ]:
proxy_geo_gdf = proxy_geo.copy()

In [ ]:
proxy_geo_gdf["geometry"] = proxy_geo_gdf["geometry"].apply(lambda x: loads(x))
proxy_geo_gdf = gpd.GeoDataFrame(proxy_geo_gdf, geometry="geometry", crs="EPSG:4326")

In [ ]:
proxy_geo_gdf.to_crs("EPSG:3857", inplace=True)

In [ ]:
proxy_geo_gdf["area_ha"] = proxy_geo_gdf.geometry.area / 10_000

In [ ]:
geo_profile = (
    proxy_geo_gdf[["Litho", "TENEMENT_N", "area_ha"]]
    .groupby(["Litho", "TENEMENT_N"])
    .sum()
)

In [ ]:
geo_profile_mng = geo_profile.join(tenement_area, lsuffix="_geotype")

In [ ]:
geo_profile_mng["perc_geotype"] = (
    geo_profile_mng["area_ha_geotype"] / geo_profile_mng["area_ha"]
) * 100

In [ ]:
geo_profile_mng.reset_index(inplace=True)

In [ ]:
geo_profile_mng = (
    geo_profile_mng.pivot(index="TENEMENT_N", columns="Litho", values="perc_geotype")
    .fillna(0)
    .reset_index()
)

## Strata

In [ ]:
strata_gdf.to_crs("EPSG:4326", inplace=True)

In [ ]:
strata_df = strata_gdf.copy()

In [ ]:
strata_df["geometry"] = strata_df["geometry"].to_wkt()

In [ ]:
strata_df.head(2)

In [ ]:
query = """
CREATE OR REPLACE TABLE strata_type AS 
SELECT 
    *EXCLUDE(geometry),
    ST_GeomFromText(geometry) as geometry FROM strata_df
"""

db.execute(query)

In [ ]:
query = """ 
CREATE OR REPLACE TABLE proxy_strata AS
SELECT
    uid,
    CADT_No,
    Davao_CADT_No,
    HOLDER,
    TENEMENT_N, 
    DATE_FILED, 
    DATE_APPRO, 
    COMMODITY, 
    REMARKS_2,
    EXPIRYDATE,
    strata,
    ST_INTERSECTION(proxy.geometry, strata_type.geometry) as geometry
FROM
    proxy
JOIN 
    strata_type
ON 
    ST_INTERSECTS(proxy.geometry, strata_type.geometry)"""

db.execute(query).df()

In [ ]:
query = """ 
SELECT
   *EXCLUDE(geometry),
   ST_astext(geometry) as geometry
FROM
    proxy_strata
"""

proxy_strata = db.execute(query).df()

In [ ]:
proxy_strata.head(2)

In [ ]:
proxy_strata_gdf = proxy_strata.copy()

In [ ]:
proxy_strata_gdf["geometry"] = proxy_strata_gdf["geometry"].apply(lambda x: loads(x))
proxy_strata_gdf = gpd.GeoDataFrame(
    proxy_strata_gdf, geometry="geometry", crs="EPSG:4326"
)

In [ ]:
proxy_strata_gdf.to_crs("EPSG:3857", inplace=True)

In [ ]:
proxy_strata_gdf["area_ha"] = proxy_strata_gdf.geometry.area / 10_000

In [ ]:
strata_profile = (
    proxy_strata_gdf[["strata", "TENEMENT_N", "area_ha"]]
    .groupby(["strata", "TENEMENT_N"])
    .sum()
)

In [ ]:
strata_profile_mng = strata_profile.join(tenement_area, lsuffix="_strata")

In [ ]:
strata_profile_mng["perc_strata"] = (
    strata_profile_mng["area_ha_strata"] / strata_profile_mng["area_ha"]
) * 100

In [ ]:
strata_profile_mng.reset_index(inplace=True)

In [ ]:
strata_profile_mng

In [ ]:
strata_profile_mng = strata_profile_mng.pivot(
    index="TENEMENT_N", columns="strata", values="perc_strata"
).fillna(0)

In [ ]:
strata_profile_mng.reset_index(inplace=True)

## HRP

In [ ]:
hrp.FOR_NFOR.unique()

In [ ]:
hrp_df = hrp.copy()

In [ ]:
hrp_df["geometry"] = hrp["geometry"].to_wkt()

In [ ]:
hrp_df.head(2)

In [ ]:
hrp_df.reset_index(inplace=True)

In [ ]:
query = """
CREATE OR REPLACE TABLE hrp AS 
SELECT 
    *EXCLUDE(geometry),
    ST_GeomFromText(geometry) as geometry FROM hrp_df
"""

db.execute(query)

In [ ]:
query = """ 
CREATE OR REPLACE TABLE proxy_hrp AS
SELECT
    uid,
    CADT_No,
    Davao_CADT_No,
    HOLDER,
    TENEMENT_N, 
    DATE_FILED, 
    DATE_APPRO, 
    COMMODITY, 
    REMARKS_2,
    EXPIRYDATE,
    FOR_NFOR,
    ST_INTERSECTION(proxy.geometry, hrp.geometry) as geometry
FROM
    proxy
JOIN 
    hrp
ON 
    ST_INTERSECTS(proxy.geometry, hrp.geometry)"""

db.execute(query).df()

In [ ]:
query = """ 
SELECT
   *EXCLUDE(geometry),
   ST_astext(geometry) as geometry
FROM
    proxy_hrp
"""

proxy_hrp = db.execute(query).df()

In [ ]:
proxy_hrp.head(2)

In [ ]:
proxy_hrp_gdf = proxy_hrp.copy()

In [ ]:
proxy_hrp_gdf["geometry"] = proxy_hrp_gdf["geometry"].apply(lambda x: loads(x))
proxy_hrp_gdf = gpd.GeoDataFrame(proxy_hrp_gdf, geometry="geometry", crs="EPSG:4326")

In [ ]:
proxy_hrp_gdf.to_crs("EPSG:3857", inplace=True)

In [ ]:
proxy_hrp_gdf["area_ha"] = proxy_hrp_gdf.geometry.area / 10_000

In [ ]:
hrp_profile = (
    proxy_hrp_gdf[["FOR_NFOR", "TENEMENT_N", "area_ha"]]
    .groupby(["FOR_NFOR", "TENEMENT_N"])
    .sum()
)

In [ ]:
hrp_profile_mng = hrp_profile.join(tenement_area, lsuffix="_fornfor")

In [ ]:
hrp_profile_mng["perc_fornfor"] = (
    hrp_profile_mng["area_ha_fornfor"] / hrp_profile_mng["area_ha"]
) * 100

In [ ]:
hrp_profile_mng.reset_index(inplace=True)

In [ ]:
hrp_profile_mng

In [ ]:
hrp_profile_mng = hrp_profile_mng.pivot(
    index="TENEMENT_N", columns="FOR_NFOR", values="perc_fornfor"
).fillna(0)

In [ ]:
hrp_profile_mng.reset_index(inplace=True)

In [ ]:
hrp_profile_mng

# Get pivot table

In [ ]:
slope_profile_mng

In [ ]:
slope_profile_mng.rename(columns={0: "below_15", 1: "above_15"}, inplace=True)

In [ ]:
elev_profile_mng

In [ ]:
elev_profile_mng.rename(
    columns={1: "below_500", 2: "500_1000", 3: "1000_1500", 4: "above_1500"},
    inplace=True,
)

In [ ]:
geo_profile_mng

In [ ]:
strata_profile_mng

In [ ]:
pivot_df = (
    proxy.merge(strata_profile_mng, how="left")
    .merge(geo_profile_mng, how="left")
    .merge(elev_profile_mng, how="left")
    .merge(slope_profile_mng, how="left")
    .merge(hrp_profile_mng, how="left")
)

In [ ]:
pivot_df.columns

In [ ]:
pivot_df.to_csv(GPKG_DATA_DIR / "tenement_percentage_profile.csv")

In [ ]:
pivot_df = pd.read_csv(GPKG_DATA_DIR / "tenement_percentage_profile.csv")

In [ ]:
strata_1 = (pivot_df["pre_strata_1"] >= 40.88) & (pivot_df["pre_strata_1"] <= 61.32)

In [ ]:
strata_2 = (pivot_df["pre_strata_2"] >= 24.88) & (pivot_df["pre_strata_2"] <= 37.32)

In [ ]:
strata_3 = (pivot_df["pre_strata_3"] >= 5.92) & (pivot_df["pre_strata_3"] <= 8.88)

In [ ]:
pivot_df[strata_1 & strata_2 & strata_3]

# Drafts

In [ ]:
hrp.reset_index(inplace=True)
hrp.rename(columns={"index": "index_hrp1"}, inplace=True)

In [ ]:
hrp.head(2)

In [ ]:
slope_reclass.reset_index(inplace=True)
slope_reclass.rename(columns={"DN": "DN_slope", "index": "index_slope"}, inplace=True)

In [ ]:
slope_reclass.head()

In [ ]:
elev_reclass.reset_index(inplace=True)
elev_reclass.rename(columns={"DN": "DN_elev", "index": "index_elev"}, inplace=True)

In [ ]:
elev_reclass.head(2)

In [ ]:
geo.reset_index(inplace=True)
geo.rename(columns={"index": "index_geotype"}, inplace=True)

In [ ]:
geo.columns

In [ ]:
geotype = geo[["index_geotype", "Litho", "remarks", "geometry"]].copy()

In [ ]:
geotype.head(2)

In [ ]:
strata_gdf.reset_index(inplace=True)
strata_gdf.rename(columns={"index": "index_strata"}, inplace=True)

In [ ]:
strata_type = strata_gdf[["index_strata", "strata", "geometry"]].copy()

In [ ]:
strata_type.head(2)

In [ ]:
union_result = gpd.overlay(
    proxy, slope_reclass, how="intersection", keep_geom_type=False
)

In [ ]:
union_result = gpd.overlay(
    union_result, elev_reclass, how="intersection", keep_geom_type=False
)

In [ ]:
union_result.to_csv(GPKG_DATA_DIR / "union_table_slope_elev.csv")

In [ ]:
union_result = union_result[
    union_result.geometry.type.isin(["Polygon", "MultiPolygon", "GeometryCollection"])
]

In [ ]:
union_result = gpd.read_file(GPKG_DATA_DIR / "BL-PL" / "union_elev_slope.gpkg")

In [ ]:
union_result.columns

In [ ]:
union_result.drop(
    columns=[
        "Remarks",
        "field_1",
        "fid_2",
    ],
    inplace=True,
)

In [ ]:
union_result.to_file(GPKG_DATA_DIR / "BL-PL" / "union_geo_slope_elev.gpkg")

In [ ]:
union_result = gpd.read_file(GPKG_DATA_DIR / "BL-PL" / "union_geo_slope_elev.gpkg")

In [ ]:
union_result.shape

In [ ]:
%%time
# List of GeoDataFrames
gdfs = [strata_type, hrp]
for gdf in tqdm(gdfs):
    union_result = gpd.overlay(union_result, gdf, how="union")

In [ ]:
union_result.geometry.type.unique()

In [ ]:
union_result.to_csv(GPKG_DATA_DIR / "union_table_all.csv")

# Create Grid

In [ ]:
from geowrangler import grids

In [ ]:
aoi = proxy.to_crs("EPSG:3857").copy()

In [ ]:
grid_generator5k = grids.SquareGridGenerator(5_000)

In [ ]:
grid_gdf5k = grid_generator5k.generate_grid(aoi)

In [ ]:
grid_gdf5k.plot()

In [ ]:
grid_gdf5k.to_crs("EPSG:4326", inplace=True)

In [ ]:
grid_gdf5k_df = grid_gdf5k.copy()

In [ ]:
grid_gdf5k_df["geometry"] = grid_gdf5k_df["geometry"].to_wkt()

In [ ]:
query = """
CREATE OR REPLACE TABLE grid_5k AS 
SELECT 
    *EXCLUDE(geometry),
    ST_GeomFromText(geometry) as geometry FROM grid_gdf5k_df
"""

db.execute(query)

In [ ]:
query = """ 
CREATE OR REPLACE TABLE proxy_grid AS 
SELECT
    uid,
    x,
    y,
    CADT_No,
    Davao_CADT_No,
    HOLDER,
    TENEMENT_N, 
    DATE_FILED, 
    DATE_APPRO, 
    COMMODITY, 
    REMARKS_2,
    EXPIRYDATE,
    ST_INTERSECTION(proxy.geometry, grid_5k.geometry) as geometry
FROM
    proxy 
JOIN 
    grid_5k
ON 
    ST_INTERSECTS(proxy.geometry, grid_5k.geometry)"""

db.execute(query).df()

In [ ]:
query = """ 
SELECT
   *EXCLUDE(geometry),
   ST_astext(geometry) as geometry
FROM
    proxy_grid 
"""

proxy_grid = db.execute(query).df()

In [ ]:
proxy_grid.head()

In [ ]:
proxy_grid_gdf = proxy_grid.copy()

In [ ]:
proxy_grid_gdf["geometry"] = proxy_grid_gdf["geometry"]

In [ ]:
proxy_grid_gdf["geometry"] = proxy_grid_gdf["geometry"].apply(lambda x: loads(x))
proxy_grid_gdf = gpd.GeoDataFrame(proxy_grid_gdf, geometry="geometry", crs="EPSG:4326")

In [ ]:
proxy_grid_gdf.explore()

In [ ]:
proxy_grid_gdf.drop_duplicates(subset=["geometry"], inplace=True)